# Virtual Proxy

> Masquerading as another object that may not even exist

A **virtual proxy** is a proxy that appears to be a fully initialized object, but it's just mimicking the underlying functionality without necessarily having it (yet). So, for all intents and purposes, it appears to be the object it's supposed to represent, but behind the scenes can offer additional functionality and behave differently.

For this example we will imagine that we're working on a photo manipulation program. We will define a `Bitmap` class that represents our images:

In [1]:
class Bitmap:
    def __init__(self, filename):
        self.filename = filename
        print(f'Loading image from {filename}')

    def draw(self):
        print(f'Drawing image {self.filename}')

We will also define an additional function for drawing images:

In [2]:
def draw_image(image):
    print('About to draw image')
    image.draw()
    print('Done drawing image')

And here's how we would use the code:

In [3]:
bmp = Bitmap('facepalm.jpg')
draw_image(bmp)

Loading image from facepalm.jpg
About to draw image
Drawing image facepalm.jpg
Done drawing image


This code seems to work fine, but let's imagine for a second that for whatever reason we comment the `draw_image(bmp)` line and do nothing else:

In [4]:
bmp = Bitmap('facepalm.jpg')
#draw_image(bmp)

Loading image from facepalm.jpg


We're still loading the full image even though we do nothing with it. This is a potentially expensive operation that we may want to avoid if our code isn't actually doing anything useful with the image.

So, how can we avoid loading the image if we're not drawing it?
1. Go back to the `Bitmap` class and modify it?
    * Open-Closed Principle violation!
2. Build a **virtual proxy** instead in order to get the API and the behavior we want.
    * Our image will only load when we actually draw it. This is called *lazy loading*.

Let's build our proxy:

In [7]:
class LazyBitmap:
    def __init__(self, filename):
        self.filename = filename
        self._bitmap = None # Private underlying class placeholder

    def draw(self):
        if not self._bitmap:
            self._bitmap = Bitmap(self.filename) # we load only when we draw!
        self._bitmap.draw()

With our virtual proxy in place, we can now lazily load images and actually load them only when we draw! Let's see it in action:

In [8]:
bmp = LazyBitmap('facepalm.jpg')  # Bitmap
draw_image(bmp)
draw_image(bmp)

About to draw image
Loading image from facepalm.jpg
Drawing image facepalm.jpg
Done drawing image
About to draw image
Drawing image facepalm.jpg
Done drawing image


Note that even though we draw the image twice, we only load it once! The virtual proxy only loads the image on first invocation, and subsequently, whenever somebody wants to draw it, it no longer performs the initialization.